Решение задачи можно разделить на две части: подготовку данных и непосредственно вычисления.
1. Подготовка данных.Поскольку данные по условию подаются как многострочная строка -- будет удобнее сначала её распарсить на отдельные сущности и дальше взаимодействовать с ними.
    1. Для начала получим отдельным числом количество доливов. Для этого найдём индекс вхождения первого символа переноса строки, затем приведём всё, что до, в число. Сложность этой операции можно оценить так: в худшем случае потребуется O(len(input_str)) операций (что в целом маловероятно из-за характера входных данных. При большой длине числа доливов, "хвост" из описания самих доливов должен быть много больше)
    2. Далее из оставшейся части соберем список пар (кортежей), в какой момент времени сколько литров долили. Для этого разделим все подстроки с парами по символу переноса строки и затем, используя list comprehension, map для преобразования типов (разбив предварительно подстроки на отдельные строки-числа) и tuple для преобразования итераторов в кортежи. Посчитаем теперь сложность этой составной операции. Срез строки после первого символа новой строки -- O(len(input_str)); разбиение строки на подстроки -- O(len(input_str)); list comprehension -- O(N), N -- количество пар (доливов); разбиение подстроки на пары по пробелу -- O(len(подстрока)); map(int, пара чисел в виде строк) -- O(1); преодразование итератора в кортеж -- O(1). В общем получаем O(N) * O(len(подстрока)) = O(N + len(подстрока)) = O(len(input_str))
    3. Таким образом алгоритмическая сложность преобразования данных -- O(len(input_str)), len(input_str) -- длина многострочной строки, поданной на вход.
2. Непосредственно расчёт.
    1. Можно было воспользоваться "наивным" алгоритмом и считать утечку и доливы каждый временной тик, вне зависимости от наличия долива, но в ситуации, когда на большое количество тиков времени (ну допустим 10к), небольшое количество доливов -- пришлось бы много шагов в цикле. А нам это не нужно -- алгоритмическая сложность такого алгоритма будет зависеть не от количества доливов, а от количества времени.
   2. Поэтому было принято решение проводить расчётные итерации только каждый долив, считая при этом, сколько времени прошло между соседними доливами. Алгоритм можно описать так:

        В цикле по количеству доливов на каждом шаге:
            проверяем, был ли предыдущий долив (позволяет учесть первый долив). Если не было -- значит, за текущий объем в увлажнителе принимаем доливаемый на данном шаге. Если был, то считаем время между текущим и предыдущим доливом (для вычисления утечки); считаем оставшееся количество воды после утечки (не может быть меньше 0); вычисляем текущее количество воды сразу после долива.

        Сложность данного алгоритма составляет O(N), N -- количество доливов
3. Суммарно сложность всей функции O(len(input_str)) + O(N) = O(len(input_str)) + O(N). Сложность расчёта без учёта обработки данных -- O(N)

In [4]:
def humidifier_calc(input_str):
    """ Function to calculate the remaining volume of water """

    """data preparation"""
    # the count of top-ups
    first_newline_index = input_str.find("\n")
    N = int(input_str[:first_newline_index])

    # top-ups list of pairs (T_i, V_i)
    top_ups = [tuple(map(int, pair.split())) for pair in input_str[(first_newline_index + 1):].splitlines()]

    """calculation"""
    # index of the previous top-up; -1 means there was no top-ups yet
    j = -1
    # volume of the water in specific moment of time
    V = 0

    for i in range(N):
        if j != -1:
            time_between_top_ups = top_ups[i][0] - top_ups[j][0]
            water_after_lack = V - time_between_top_ups if V - time_between_top_ups >= 0 else 0
            V = water_after_lack + top_ups[i][1]
        else:
            V = top_ups[i][1]
        j = i

    return str(V)


Тесты:

In [5]:
import pytest


@pytest.mark.parametrize(
    "test_input,expected", [("4\n1 3\n3 1\n4 4\n7 1", "3"),
                            ("3\n1 8\n10 11\n21 5", "5"),
                            ("10\n2 1\n22 10\n26 17\n29 2\n45 20\n47 32\n72 12\n75 1\n81 31\n97 7", "57")]
)
def test_humidifier(test_input, expected):
    assert humidifier_calc(test_input) == expected
